# Journal de bord

Copier-coller les cellules comme canevas pour y remplir les informations.

## 📅 Journal de bord – Jour 1 (17/12/2025)

### 🔧 Étapes / Actions réalisées
- Découverte du sujet,
- Récupération des ressources,
- Structuration des dossiers
- Mise en place du tracking **git**

**Première approche de découpage des tâches**

**Traitement des fichiers de données tabulaires**
1. Analyser les fichiers de données et définir la stratégie de nettoyage préalable : données sensibles, données personnelles, données interdites,
2. Procéder à la fusion des jeux de données en incluant une information supplémentaire pour ne pas perdre l'origine de l'information
3. Produire dans outputs/data_processed le fichier csv résultant

## 📅 Journal de bord – Jour 2 (18/12/2025)

### 🔧 Étapes / Actions réalisées

- Mise en attente des processus de détection des NaN et Outliers, j'ajouterai les handlers associés plus tard en mettant à jour le pipeline,
- Pour faciliter l'entraînement des modèles selon : deux algorithmes et 4 scénarios, je vais mettre en place un pattern *Strategy* ; concrètement ce pattern me permettra de rejouer le même handler en changeant dynamiquement la stratégie d'entraînement. Le principe DRY (Don't Repeat Yourself) oblige à réfléchir à la manière de ne pas réécrire plusieurs fois la même chose,
- Les deux stratégies : Logistic Regression (LR) et Random Forest (RF)

**Note** : Intégration de MLFlow et des artefacts, paramètres (hyper paramètres) qui permettra de choisir le modèle le plus performant et facilitant la lecture des performances globales.

### 📝 Observations / Difficultés
- 

### ✅ Prochaines étapes
- 